In [72]:
import csv 
import time
import requests
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
SCROLL_PAUSE_TIME = 0.5


URL = 'https://the-sleeper.com/en/type-men/pajamas-en/'
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(URL)


# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


soup = BeautifulSoup(driver.page_source,'html.parser')

products = []
for i in soup.find_all('div',{'class':'gi-title'}):
    products.append((str(i.text).strip()))


Looking for [chromedriver 86.0.4240.22 mac64] driver in cache 
File found in cache by path [/Users/mandili/.wdm/drivers/chromedriver/86.0.4240.22/mac64/chromedriver]


In [74]:
product_links = []
for i in soup.find_all('a',{'class':'vd-grid-text'}):
    product_links.append(i.get('href'))
print(len(product_links))

11


In [75]:
original_prices = []
for i in soup.find_all('div',{'class':'gi-price'}):
    if i.find('del') != None:
        original_prices.append(str(i.find('del').find('span').text).replace('\xa0$',''))
    else:
        original_prices.append(None)

In [76]:
current_prices = []
for i in soup.find_all('div',{'class':'gi-price'}):
    if(i.find('ins')) == None:
        current_prices.append(str(i.find('span').text).strip().replace('\xa0$',''))
    else:
        current_prices.append(str(i.find('ins').span.text).replace('\xa0$',''))

In [77]:
pics = []
for i in soup.find_all('div',{'class':'gi-img'}):
    pics.append(i.img.get('data-src'))

In [79]:
import os
import requests as req

os.mkdir('photos')
i = 1
for index, link in enumerate(pics):
    img_data = req.get(link).content
    with open('photos/'+str(index+1)+'.png','wb+') as f:
        f.write(img_data)
    i+=1

In [80]:
import pandas as pd

column_names = ["product", "original price", 'current price', 'product link', 'picture link']
df = pd.DataFrame(columns = column_names)

df['product'] = products
df['original price'] = original_prices
df['current price'] = current_prices
df['product link'] = product_links
df['picture link'] = pics
df.to_csv('sleep-men-pj.csv')

In [73]:
products

['Cream Linen Unisex Pajama Set with Pants',
 'Azure Linen Unisex Pajama Set with Pants',
 'Coal Black Unisex Pajama Set with Pants',
 'Unisex Sepia Linen Pajama Set with Pants',
 'Navy Unisex Pajama Set with Pants',
 'Cream Linen Unisex Pajama Set with Shorts',
 'Azure Linen Unisex Pajama Set with Shorts',
 'Unisex Coal Black Linen Pajama Set with Shorts',
 'Unisex Navy Linen Pajama Set with Shorts',
 'Unisex Sepia Linen Pajama Set with Shorts',
 'Frank Underwood Pajama Set with Pants']